In [1]:
# Import modules
import ee
import geemap
import geopandas as gpd
import numpy as np

In [2]:
# Initialize Earth Engine
try:
    ee.Initialize()
except ee.EEException:
    ee.Authenticate()

In [7]:
# Open source table as a dataframe
country_gpkg = r"data\zambia_boundary.gpkg"
country_gdf = gpd.read_file(country_gpkg)

# Preview the data
# country_gdf.head()

In [ ]:
# Get images for rainy season (Nov to Mar) only
def get_rain_season_images(rainfall_dataset, start_year, end_year):
    # Create empty image collection
    merged_image_collection = ee.ImageCollection([])
    
    for year in range(start_year, end_year):
        # Nov to Dec of current year
        start1 = ee.Date(f'{year}-11-01')
        end1 = ee.Date(f'{year}-12-31')
        season_part1 = rainfall_dataset.filterDate(start1, end1)

        # Jan to Mar of following year
        start2 = ee.Date(f'{year + 1}-01-01')
        end2 = ee.Date(f'{year + 1}-03-31')
        season_part2 = rainfall_dataset.filterDate(start2, end2)

        # Merge image collection
        merged_year_collection = season_part1.merge(season_part2)
        merged_image_collection = merged_image_collection.merge(merged_year_collection)

    return merged_image_collection

In [128]:
def get_avg_rainfall(image_collection, aoi, start_year, end_year):
    
    no_years = end_year - start_year

    # Get images for rainfal season between selected dates
    seasonal_rainfall_dataset = get_rain_season_images(image_collection, start_year, end_year)

    # Clip images to AOI and compute the mean
    avg_rainfall = seasonal_rainfall_dataset.sum() \
                    .divide(no_years) \
                    .clip(aoi)

    return avg_rainfall

In [276]:
def get_crop_suitable_rainfall_image(crop, image_collection, aoi, start_year, end_year):
    ideal_crop_rainfall = {
        'Soybean': [600, 1000],
        'Groundnut': [500, 1000],
        'Sugarbean': [500, 1000],
        'Navybean': [500, 1000],
        'Maize': [600, 1200],
        'Cowpea': [400, 800],
        'Sunflower': [400, 800]
    }

    # Clean input
    crop = crop.strip().replace(" ","").title()

    # Get average rainfall
    avg_rainfall = get_avg_rainfall(image_collection, aoi, start_year, end_year)

    # Mask out crop suitable rainfall
    crop_suitable_rainfall_masked = avg_rainfall \
        .updateMask(avg_rainfall \
        .gte(ideal_crop_rainfall[crop][0]) \
        .And(avg_rainfall.lte(ideal_crop_rainfall[crop][1]))
        )
    
    return crop_suitable_rainfall_masked

In [287]:
def get_vis_params(image, aoi):
    # reduceRegion to get the minimum avg rainfall in the AOI
    min_rainfall = image.reduceRegion(
        reducer=ee.Reducer.min(),
        geometry=aoi,
        scale=5566,  # CHIRPS dataset native resolution
        maxPixels=1e8
    )

    # reduceRegion to get the maximum avg rainfall in the AOI
    max_rainfall = image.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=aoi,
    scale=5566,  # CHIRPS dataset native resolution
    maxPixels=1e8
    )

    # Get the result and print the minimum value
    try:
        rainfall_min_max = [min_rainfall.getInfo()['precipitation'], 
                            max_rainfall.getInfo()['precipitation']]
        vis_params = {'palette': ['darkred', 'red', 
                          'white', 'lightblue', 'blue', 'darkblue'],
               'min': rainfall_min_max[0], 'max': rainfall_min_max[1]}
        
    except KeyError:
        rainfall_min_max = [min_rainfall.getInfo()['constant'], 
                            max_rainfall.getInfo()['constant']]
        vis_params = {'palette': ['darkred', 'yellow', 'green', 'darkgreen'],
                'min': rainfall_min_max[0], 'max': rainfall_min_max[1]}
    
    return vis_params

In [288]:
# Get weighted rainfall suitability
def ranked_rainfall_suitability_image(crop, image, aoi):
    rainfall_suitability_ranges = {
        'Soybean': [600, 800, 900, 1000],
        'Groundnut': [500, 600, 900, 1000],
        'Sugarbean': [500, 600, 900, 1000],
        'Navybean': [500, 600, 900, 1000],
        'Maize': [600, 800, 1000, 1200],
        'Cowpea': [400, 550, 700, 800],
        'Sunflower': [400, 550, 700, 800]
    }

    # Clean input
    crop = crop.strip().replace(" ","").title()

    # Assign suitability values based on rainfall ranges
    suitability_image = ee.Image(0).clip(aoi)  # Default suitability is 0 (no suitability)

    # Assign 1, low suitability
    suitability_image = suitability_image.where(image.gte(rainfall_suitability_ranges[crop][0]) \
                                                .And(image.lt(rainfall_suitability_ranges[crop][1])), 1)
    
    # Assign 2, medium suitability
    suitability_image = suitability_image.where(image.gte(rainfall_suitability_ranges[crop][1]) \
                                                .And(image.lt(rainfall_suitability_ranges[crop][2])), 2)

    # Assign 3, high suitability
    suitability_image = suitability_image.where(image.gte(rainfall_suitability_ranges[crop][2]), 3)
    
    return suitability_image

In [289]:
# Convert county from gdf to ee
country_ee = geemap.gdf_to_ee(country_gdf)

# Import rainfall dataset from GEE
rainfall_dataset = ee.ImageCollection("UCSB-CHG/CHIRPS/PENTAD") \
            .select('precipitation') 

start_year, end_year = 2019, 2025

crop = "Maize"

#  Get average rainfall
avg_rainfall = get_avg_rainfall(rainfall_dataset, country_ee, start_year, end_year)

# Get crop suitable average rainfall
suitable_rainfall = get_crop_suitable_rainfall_image(crop, rainfall_dataset, country_ee, start_year, end_year)

# Get ranked crop suitable average rainfall
ranked_suitable_rainfall = ranked_rainfall_suitability_image(crop, suitable_rainfall, country_ee)


In [290]:
# Rainfall visualisastion parameters
avg_rainfall_vis_params = get_vis_params(avg_rainfall, country_ee)
suitable_rainfall_vis_params = get_vis_params(suitable_rainfall, country_ee)
ranked_rainfall_vis_params = get_vis_params(ranked_suitable_rainfall, country_ee)

In [292]:
# Visualise layers
m = geemap.Map()
m.zoom_to_gdf(country_gdf)

# m.add_layer(country_ee, name= 'Zambia')
# m.add_ee_layer(avg_rainfall, vis_params= avg_rainfall_vis_params, name= "Rainfall")
# m.add_ee_layer(suitable_rainfall, vis_params= suitable_rainfall_vis_params, name= f"{crop} Suitable Areas")

m.add_ee_layer(ranked_suitable_rainfall, vis_params= ranked_rainfall_vis_params, name= f"{crop} Suitable Areas")
m.add_colorbar(ranked_rainfall_vis_params, orientation='vertical',  
               label= "Rainfall Suitability", layer_name= f"{crop} Rainfall Suitability")
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…